In [ ]:
import json
with open(r"train_qa.json","r") as read_file:
  train=json.load(read_file)

In [ ]:
train=train['data']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
arrayForDF=[]
for current_subject in train:
  # print(current_subject)
  subject = current_subject['title']
  for current_context in current_subject['paragraphs']:
    context = current_context['context'] 
    for current_question in current_context['qas']:
      question = current_question['question']
      if len(question) > 2:
        is_impossible = current_question['is_impossible']
        if is_impossible == False:
          for answer in current_question['answers']:
            answer_text = answer["text"]
            answer_start = answer["answer_start"]

            recode = {
                "answer_text":answer_text,
                "answer_start":answer_start,
                "question":question,
                "context":context,
                "subject":subject
            }

            arrayForDF.append(recode)

train_df = pd.DataFrame(arrayForDF)

train_df.head()


In [ ]:
for i in range(0,len(train_df['question'])):
  que=train_df['question'][i].lower()
  if(que.find('why')!=-1 or que.find('how')!=-1):
    train_df['question'][i]=np.nan

In [ ]:
for i in range(0,len(train_df['answer_text'])):
  ans=str(train_df['answer_text'][i].lower())
  if(len(ans.split())>5):
    train_df['answer_text'][i]=np.nan

In [ ]:
for i in range(0,len(train_df['context'])):
  con=str(train_df['context'][i].lower())
  if(len(con.split())>50):
    train_df['context'][i]=np.nan

In [ ]:
train_df.dropna(inplace=True)
train_df=train_df.reset_index()
train_df

In [ ]:
len(train_df)

#QA

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM, GRU
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

from functools import reduce
import tarfile
import numpy as np
import re

import IPython
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def tokenize(sent):
    return [ x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [ ]:
def parse_stories(lines, only_supporting=False):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [ ]:
def get_stories(f, only_supporting=False, max_length=None):
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [ ]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [ ]:
train_df

,index,answer_text,answer_start,question,context,subject
0,315,Tidal,140,"Beyonce released the song ""Formation"" on which...","On February 6, 2016, one day before her perfor...",Beyoncé
1,316,Formation,154,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor...",Beyoncé
2,317,"February 6, 2016",3,"What day did Beyonce release her single, Forma...","On February 6, 2016, one day before her perfor...",Beyoncé
3,319,Tidal,140,What was the name of the streaming service?,"On February 6, 2016, one day before her perfor...",Beyoncé
4,320,music streaming,116,What kind of platform was the song released?,"On February 6, 2016, one day before her perfor...",Beyoncé
...,...,...,...,...,...,...
3347,23215,1990,118,"when was madonna's hit single ""vogue"" released?",The early 1990s additionally saw the rise in m...,House_music
3348,23216,1990s,10,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...,House_music
3349,23217,Madonna,108,"who sang the hit single ""vogue""?",The early 1990s additionally saw the rise in m...,House_music
3350,23261,its roots,110,There has been a reunification of contemporary...,In addition to these popular examples of house...,House_music


In [ ]:
train_df.drop(columns=['answer_start','subject','index'],inplace=True)

In [ ]:
train_df

,answer_text,question,context
0,Tidal,"Beyonce released the song ""Formation"" on which...","On February 6, 2016, one day before her perfor..."
1,Formation,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor..."
2,"February 6, 2016","What day did Beyonce release her single, Forma...","On February 6, 2016, one day before her perfor..."
3,Tidal,What was the name of the streaming service?,"On February 6, 2016, one day before her perfor..."
4,music streaming,What kind of platform was the song released?,"On February 6, 2016, one day before her perfor..."
...,...,...,...
3347,1990,"when was madonna's hit single ""vogue"" released?",The early 1990s additionally saw the rise in m...
3348,1990s,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...
3349,Madonna,"who sang the hit single ""vogue""?",The early 1990s additionally saw the rise in m...
3350,its roots,There has been a reunification of contemporary...,In addition to these popular examples of house...


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train_df, test_size=0.2)

In [ ]:
train

,answer_text,question,context
3081,The Great British Bake Off,"Nadiya Hussain, a Times Magazine column writer...",The Times Magazine features columns touching o...
1494,in the 1980s,When did Chen Fangyun come up with the idea fo...,The original idea of a Chinese satellite navig...
1336,coffee,Which product of Saint Helena was praised by N...,Owing to Napoleon's praise of Saint Helena’s c...
2492,convert to Catholicism,What social pressure was pressing down the peo...,The Orthodox nobles' rights were nominally equ...
463,Beautiful Life,What was the name of Nick Fradianis victory song?,"Nick Fradiani won the season, defeating Clark ..."
...,...,...,...
1268,Irish pubs,What is an example of a theme pub that caters ...,"Pubs that cater for a niche clientele, such as..."
3023,In the 18th century,When did the Black Guard exist?,"In the 18th century, the Moroccan Sultan Moula..."
587,1.6%,What is the percentage of Muslims living in th...,"According to CIA World Factbook, the people of..."
1985,academic computing,What division of the University of Kansas cont...,KU's academic computing department was an acti...


In [ ]:
train=train[train.columns[::-1]]
test=test[test.columns[::-1]]

In [ ]:
import gensim
from gensim.models import Word2Vec
train['context']=train['context'].apply(lambda x: gensim.utils.simple_preprocess(x))
train['question']=train['question'].apply(lambda x: gensim.utils.simple_preprocess(str(x)))

In [ ]:
train

,context,question,answer_text
3081,"[the, times, magazine, features, columns, touc...","[nadiya, hussain, times, magazine, column, wri...",The Great British Bake Off
1494,"[the, original, idea, of, chinese, satellite, ...","[when, did, chen, fangyun, come, up, with, the...",in the 1980s
1336,"[owing, to, napoleon, praise, of, saint, helen...","[which, product, of, saint, helena, was, prais...",coffee
2492,"[the, orthodox, nobles, rights, were, nominall...","[what, social, pressure, was, pressing, down, ...",convert to Catholicism
463,"[nick, fradiani, won, the, season, defeating, ...","[what, was, the, name, of, nick, fradianis, vi...",Beautiful Life
...,...,...,...
1268,"[pubs, that, cater, for, niche, clientele, suc...","[what, is, an, example, of, theme, pub, that, ...",Irish pubs
3023,"[in, the, th, century, the, moroccan, sultan, ...","[when, did, the, black, guard, exist]",In the 18th century
587,"[according, to, cia, world, factbook, the, peo...","[what, is, the, percentage, of, muslims, livin...",1.6%
1985,"[ku, academic, computing, department, was, an,...","[what, division, of, the, university, of, kans...",academic computing


In [ ]:
train_records=train.to_records(index=False)
train_records = list(train_records)
print(train_records[0])

(list(['the', 'times', 'magazine', 'features', 'columns', 'touching', 'on', 'various', 'subjects', 'such', 'as', 'celebrities', 'fashion', 'and', 'beauty', 'food', 'and', 'drink', 'homes', 'and', 'gardens', 'or', 'simply', 'writers', 'anecdotes', 'notable', 'contributors', 'include', 'giles', 'coren', 'food', 'and', 'drink', 'writer', 'of', 'the', 'year', 'in', 'and', 'nadiya', 'hussain', 'winner', 'of', 'bbc', 'the', 'great', 'british', 'bake', 'off']), list(['nadiya', 'hussain', 'times', 'magazine', 'column', 'writer', 'is', 'known', 'for', 'being', 'the', 'winner', 'of', 'what', 'bbc', 'contest']), 'The Great British Bake Off')


In [ ]:
print(train_records[200])

(list(['the', 'first', 'actual', 'usage', 'of', 'the', 'term', 'prime', 'minister', 'or', 'premier', 'ministre', 'citation', 'needed', 'was', 'used', 'by', 'cardinal', 'richelieu', 'when', 'in', 'he', 'was', 'named', 'to', 'head', 'the', 'royal', 'council', 'as', 'prime', 'minister', 'of', 'france', 'louis', 'xiv', 'and', 'his', 'descendants', 'generally', 'attempted', 'to', 'avoid', 'giving', 'this', 'title', 'to', 'their', 'chief', 'ministers']), list(['what', 'country', 'did', 'richelieu', 'serve', 'as', 'prime', 'minister', 'for']), 'France')


In [ ]:
test['question']=test['question'].apply(lambda x: gensim.utils.simple_preprocess(x))
test['context']=test['context'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [ ]:
test_records=test.to_records(index=False)
test_records = list(test_records)
print(test_records[0])

(list(['uncompressed', 'audio', 'as', 'stored', 'on', 'an', 'audio', 'cd', 'has', 'bit', 'rate', 'of', 'kbit', 'bit', 'sample', 'samples', 'second', 'channels', 'bits', 'kilobit', 'so', 'the', 'bitrates', 'and', 'kbit', 'represent', 'compression', 'ratios', 'of', 'approximately', 'and', 'respectively']), list(['what', 'would', 'bit', 'rate', 'of', 'kbit', 'have', 'for', 'compression', 'ratio']), '7:1')


In [ ]:
vocab = set()
for story, q, answer in train_records + test_records:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_records + test_records)))
query_maxlen = max(map(len, (x for _, x, _ in train_records + test_records)))

In [ ]:
story_maxlen,query_maxlen

(55, 30)

In [ ]:
# vocab[1]
vocab_size

11841

In [ ]:
print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_records))
print('Number of test stories:', len(test_records))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_records[0])
print('-')
print('Vectorizing the word sequences...')

-
Vocab size: 11841 unique words
Story max length: 55 words
Query max length: 30 words
Number of training stories: 2681
Number of test stories: 671
-
Here's what a "story" tuple looks like (input, query, answer):
(list(['the', 'times', 'magazine', 'features', 'columns', 'touching', 'on', 'various', 'subjects', 'such', 'as', 'celebrities', 'fashion', 'and', 'beauty', 'food', 'and', 'drink', 'homes', 'and', 'gardens', 'or', 'simply', 'writers', 'anecdotes', 'notable', 'contributors', 'include', 'giles', 'coren', 'food', 'and', 'drink', 'writer', 'of', 'the', 'year', 'in', 'and', 'nadiya', 'hussain', 'winner', 'of', 'bbc', 'the', 'great', 'british', 'bake', 'off']), list(['nadiya', 'hussain', 'times', 'magazine', 'column', 'writer', 'is', 'known', 'for', 'being', 'the', 'winner', 'of', 'what', 'bbc', 'contest']), 'The Great British Bake Off')
-
Vectorizing the word sequences...


In [ ]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx_word = dict((i+1, c) for i,c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_records,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)

In [ ]:
inputs_test, queries_test, answers_test = vectorize_stories(test_records,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

In [ ]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (2681, 55)
inputs_test shape: (671, 55)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (2681, 30)
queries_test shape: (671, 30)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (2681, 11841)
answers_test shape: (671, 11841)
-
Compiling...


In [ ]:
inputs_train.shape, queries_train.shape, answers_train.shape

((2681, 55), (2681, 30), (2681, 11841))

In [ ]:
train_epochs = 1000
batch_size = 32
lstm_size = 64

In [ ]:
story_maxlen,query_maxlen,vocab_size

(55, 30, 11841)

In [ ]:
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

print('Input sequence:', input_sequence)
print('Question:', question)

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)
response = add([match, input_encoded_c]) 
response = Permute((2, 1))(response)  
answer = concatenate([response, question_encoded])
answer = LSTM(lstm_size)(answer) 
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  
answer = Activation('softmax')(answer)

Input sequence: KerasTensor(type_spec=TensorSpec(shape=(None, 55), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Question: KerasTensor(type_spec=TensorSpec(shape=(None, 30), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")


In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 55)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, None, 64)     757824      ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 30, 64)       757824      ['input_2[0][0]']                
                                                                                              

In [ ]:
history = model.fit([inputs_train, queries_train], answers_train, epochs=train_epochs, batch_size=batch_size, verbose=1) 
model.save('/content/drive/MyDrive/qamodel.h5')

84/84 [==============================] - 5s 65ms/step - loss: 6.3281 - accuracy: 0.1264 - f1_m: 0.0300 - precision_m: 0.4048 - recall_m: 0.0156
Epoch 353/1000
49/84 [================>.............] - ETA: 2s - loss: 6.2322 - accuracy: 0.1403 - f1_m: 0.0412 - precision_m: 0.4898 - recall_m: 0.0217

KeyboardInterrupt: ignored

In [ ]:
for i in range(0,10):
        current_inp = test_records[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")

#filtering

In [ ]:
train

,context,question,answer_text
19,"[chopin, life, was, covered, in, bbc, tv, docu...","[what, two, people, created, documentary, on, ...",Angelo Bozzolini and Roberto Prosseda
469,"[nick, fradiani, won, the, season, defeating, ...","[which, contestant, came, in, third, place]",Jax
1641,"[central, catalan, is, considered, the, standa...","[what, is, central, catalan]",standard pronunciation
2255,"[in, during, the, english, civil, war, parliam...","[what, is, the, furthest, town, the, royalists...",Redbridge
1239,"[there, was, already, regulation, on, public, ...","[what, party, received, income, from, the, sal...",the crown
...,...,...,...
266,"[in, following, the, death, of, west, mother, ...","[what, was, the, kanye, west, foundation, rena...",The Dr. Donda West Foundation
189,"[in, peter, stuyvesant, the, director, general...","[in, what, year, did, the, english, take, over...",1664
1121,"[according, to, unfpa, these, elements, promot...","[unfpa, lists, elements, that, promote, what, ...",reproductive health
3047,"[the, third, john, walter, the, founder, grand...","[what, year, did, the, founder, grandson, succ...",1847


##load model

In [ ]:
dependencies = {
    'accuracy': 'accuracy',
    'f1_m':f1_m,
    'precision_m':precision_m,
    'recall_m':recall_m
}

In [ ]:
reconstructed_model = keras.models.load_model("/content/drive/MyDrive/qamodel.h5", custom_objects=dependencies)

In [ ]:
inputs_test
# queries_test
# answers_test

array([[    0,     0,     0, ...,  2166,  2073,  9255],
       [    0,     0,     0, ...,  4801,  8245,  5327],
       [    0,     0,     0, ...,  9541,  3597,  9522],
       ...,
       [    0,     0,     0, ..., 10896,  2510,  4954],
       [    0,     0,     0, ...,  2406,  9942,  5522],
       [    0,     0,     0, ...,  8798,  7872,  5442]], dtype=int32)

In [ ]:
reconstructed_model.fit([inputs_test, queries_test], answers_test)

21/21 [==============================] - 3s 62ms/step - loss: 7.6467 - accuracy: 0.0671 - f1_m: 0.0144 - precision_m: 0.1905 - recall_m: 0.0075


#what

In [ ]:
temp_df=train_df.copy()
temp_df

,answer_text,question,context
0,Tidal,"Beyonce released the song ""Formation"" on which...","On February 6, 2016, one day before her perfor..."
1,Formation,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor..."
2,"February 6, 2016","What day did Beyonce release her single, Forma...","On February 6, 2016, one day before her perfor..."
3,Tidal,What was the name of the streaming service?,"On February 6, 2016, one day before her perfor..."
4,music streaming,What kind of platform was the song released?,"On February 6, 2016, one day before her perfor..."
...,...,...,...
3347,1990,"when was madonna's hit single ""vogue"" released?",The early 1990s additionally saw the rise in m...
3348,1990s,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...
3349,Madonna,"who sang the hit single ""vogue""?",The early 1990s additionally saw the rise in m...
3350,its roots,There has been a reunification of contemporary...,In addition to these popular examples of house...


In [ ]:
# for i in range(0,len(train['question'])):
for i in range(len(temp_df['question'])):
  que=str(temp_df['question'][i])
  if(que.find('what')==-1):
    temp_df['question'][i]=np.nan

In [ ]:
temp_df

,answer_text,question,context
0,Tidal,NaN,"On February 6, 2016, one day before her perfor..."
1,Formation,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor..."
2,"February 6, 2016",NaN,"On February 6, 2016, one day before her perfor..."
3,Tidal,NaN,"On February 6, 2016, one day before her perfor..."
4,music streaming,NaN,"On February 6, 2016, one day before her perfor..."
...,...,...,...
3347,1990,NaN,The early 1990s additionally saw the rise in m...
3348,1990s,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...
3349,Madonna,NaN,The early 1990s additionally saw the rise in m...
3350,its roots,There has been a reunification of contemporary...,In addition to these popular examples of house...


In [ ]:
what_df=temp_df.dropna()
what_df=what_df.reset_index()
y_what=what_df['answer_text']
what_df.drop(['index'],axis=1,inplace=True)
what_df
what_df['context']=what_df['context'].apply(lambda x: gensim.utils.simple_preprocess(x))
what_df['question']=what_df['question'].apply(lambda x: gensim.utils.simple_preprocess(str(x)))\
what_df=what_df[what_df.columns[::-1]]
what_df_records=what_df.to_records(index=False)
what_df_records = list(what_df_records)
print(what_df_records[0])
inputs_what, queries_what, answers_what = vectorize_stories(what_df_records,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
reconstructed_model.fit([inputs_what, queries_what], answers_what)

(list(['on', 'february', 'one', 'day', 'before', 'her', 'performance', 'at', 'the', 'super', 'bowl', 'beyoncé', 'released', 'new', 'single', 'exclusively', 'on', 'music', 'streaming', 'service', 'tidal', 'called', 'formation']), list(['beyonce', 'new', 'single', 'released', 'before', 'the', 'super', 'bowl', 'was', 'called', 'what']), 'Formation')


In [ ]:
dependencies = {
    'accuracy': 'accuracy',
    'f1_m':f1_m,
    'precision_m':precision_m,
    'recall_m':recall_m
}

In [ ]:
reconstructed_model = keras.models.load_model("/content/drive/MyDrive/qamodel.h5", custom_objects=dependencies)

24/24 [==============================] - 3s 58ms/step - loss: 7.6448 - accuracy: 0.0841 - f1_m: 0.0175 - precision_m: 0.2500 - recall_m: 0.0091


#when

In [ ]:
temp_df=train_df.copy()
temp_df

,answer_text,question,context
0,Tidal,"Beyonce released the song ""Formation"" on which...","On February 6, 2016, one day before her perfor..."
1,Formation,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor..."
2,"February 6, 2016","What day did Beyonce release her single, Forma...","On February 6, 2016, one day before her perfor..."
3,Tidal,What was the name of the streaming service?,"On February 6, 2016, one day before her perfor..."
4,music streaming,What kind of platform was the song released?,"On February 6, 2016, one day before her perfor..."
...,...,...,...
3347,1990,"when was madonna's hit single ""vogue"" released?",The early 1990s additionally saw the rise in m...
3348,1990s,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...
3349,Madonna,"who sang the hit single ""vogue""?",The early 1990s additionally saw the rise in m...
3350,its roots,There has been a reunification of contemporary...,In addition to these popular examples of house...


In [ ]:
for i in range(len(temp_df['question'])):
  que=str(temp_df['question'][i])
  if(que.find('when')==-1):
    temp_df['question'][i]=np.nan

In [ ]:
when_df=temp_df.dropna()
when_df=when_df.reset_index()

In [ ]:
when_df.drop(['index'],axis=1,inplace=True)

In [ ]:
when_df['context']=when_df['context'].apply(lambda x: gensim.utils.simple_preprocess(x))
when_df['question']=when_df['question'].apply(lambda x: gensim.utils.simple_preprocess(str(x)))

In [ ]:
when_df=when_df[when_df.columns[::-1]]

In [ ]:
when_df_records=when_df.to_records(index=False)
when_df_records = list(when_df_records)
print(when_df_records[0])

(list(['in', 'october', 'beyoncé', 'signed', 'deal', 'to', 'launch', 'an', 'activewear', 'line', 'of', 'clothing', 'with', 'british', 'fashion', 'retailer', 'topshop', 'the', 'venture', 'is', 'called', 'parkwood', 'topshop', 'athletic', 'ltd', 'and', 'is', 'scheduled', 'to', 'launch', 'its', 'first', 'dance', 'fitness', 'and', 'sports', 'ranges', 'in', 'autumn', 'the', 'line', 'will', 'launch', 'in', 'april']), list(['beyonce', 'and', 'topshops', 'first', 'products', 'were', 'to', 'be', 'sold', 'in', 'stores', 'when']), 'autumn 2015')


In [ ]:
inputs_when, queries_when, answers_when = vectorize_stories(when_df_records,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)

In [ ]:
reconstructed_model.fit([inputs_when, queries_when], answers_when)

2/2 [==============================] - 0s 59ms/step - loss: 8.3401 - accuracy: 0.0435 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00


#where

In [ ]:
temp_df=train_df.copy()
temp_df

,answer_text,question,context
0,Tidal,"Beyonce released the song ""Formation"" on which...","On February 6, 2016, one day before her perfor..."
1,Formation,Beyonce's new single released before the super...,"On February 6, 2016, one day before her perfor..."
2,"February 6, 2016","What day did Beyonce release her single, Forma...","On February 6, 2016, one day before her perfor..."
3,Tidal,What was the name of the streaming service?,"On February 6, 2016, one day before her perfor..."
4,music streaming,What kind of platform was the song released?,"On February 6, 2016, one day before her perfor..."
...,...,...,...
3347,1990,"when was madonna's hit single ""vogue"" released?",The early 1990s additionally saw the rise in m...
3348,1990s,what decade did house music hit the mainstream...,The early 1990s additionally saw the rise in m...
3349,Madonna,"who sang the hit single ""vogue""?",The early 1990s additionally saw the rise in m...
3350,its roots,There has been a reunification of contemporary...,In addition to these popular examples of house...


In [ ]:
for i in range(len(temp_df['question'])):
  que=str(temp_df['question'][i])
  if(que.find('where')==-1):
    temp_df['question'][i]=np.nan

In [ ]:
where_df=temp_df.dropna()
where_df=where_df.reset_index()
where_df.drop(['index'],axis=1,inplace=True)

In [ ]:
where_df['context']=where_df['context'].apply(lambda x: gensim.utils.simple_preprocess(x))
where_df['question']=where_df['question'].apply(lambda x: gensim.utils.simple_preprocess(str(x)))
where_df=where_df[where_df.columns[::-1]]

In [ ]:
where_df_records=where_df.to_records(index=False)
where_df_records = list(where_df_records)
print(where_df_records[0])

(list(['the', 'idea', 'of', 'the', 'decline', 'and', 'gradual', 'disappearance', 'of', 'the', 'teaching', 'has', 'been', 'influential', 'in', 'east', 'asian', 'buddhism', 'pure', 'land', 'buddhism', 'holds', 'that', 'it', 'has', 'declined', 'to', 'the', 'point', 'where', 'few', 'are', 'capable', 'of', 'following', 'the', 'path', 'so', 'it', 'may', 'be', 'best', 'to', 'rely', 'on', 'the', 'power', 'of', 'amitābha']), list(['what', 'form', 'of', 'buddhism', 'has', 'declined', 'to', 'the', 'point', 'where', 'few', 'are', 'capable', 'of', 'following', 'the', 'path']), 'Pure Land')


In [ ]:
inputs_where, queries_where, answers_where = vectorize_stories(where_df_records,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)

In [ ]:
reconstructed_model.fit([inputs_where, queries_where], answers_where)

2/2 [==============================] - 0s 68ms/step - loss: 7.4580 - accuracy: 0.0882 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00


#which

In [ ]:
temp_df=train_df.copy()
temp_df